In [5]:
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
import seaborn as sns

In [6]:
class graph:
    def __init__(self, x, y, theta):
        self.x = x
        self.y = y
        self.x_points = np.array([x])
        self.y_points = np.array([y])
        self.theta = theta

    def move_forward(self, distance):
        self.x += distance * np.cos(np.radians(self.theta))
        self.y += distance * np.sin(np.radians(self.theta))
        self.x_points = np.append(self.x_points, self.x)
        self.y_points = np.append(self.y_points, self.y)

    # Function to turn by a certain angle (degrees)
    def turn(self, degrees):
        self.theta += degrees


In [7]:

def calculate_error(car, track):
    diff = np.sin(np.radians(car.theta + 90)) * (track.x_points - car.x) - np.cos(np.radians(car.theta + 90)) * (track.y_points - car.y)
    crossings = np.where(np.diff(np.sign(diff)))[0]
    if crossings.size == 0:
        return 0
    
    distances = (track.x_points[crossings] - car.x) ** 2 + (track.y_points[crossings] - car.y) ** 2
    shortest_distance_index = np.argmin(distances)
    shortest_distance = distances[shortest_distance_index]

    if np.abs(shortest_distance) > 1:
        return 0
    
    # shortest_distance_index = 0  # initialize to 0
    
    i_x1 = track.x_points[crossings[shortest_distance_index]]
    i_y1 = track.y_points[crossings[shortest_distance_index]]

    i_x2 = track.x_points[crossings[shortest_distance_index] + 1]
    i_y2 = track.y_points[crossings[shortest_distance_index] + 1]

    if (car.y - i_y1) * (i_x2 - i_x1) - (i_y2 - i_y1) * (car.x - i_x1) > 0: # if the car is on the right side of the track (use in case of 1 or 3 quadrant)
        shortest_distance = -shortest_distance
    # car_vector = np.array([np.cos(np.radians(car.theta)), np.sin(np.radians(car.theta))])
    # sensor_vector = np.array([track.x_points[shortest_distance_index] - car.x , track.y_points[shortest_distance_index] - car.y])
    # print(car_vector, sensor_vector)
    # print(np.cross(car_vector, sensor_vector))
    # if np.cross(car_vector, sensor_vector) < 0:
    #     shortest_distance = -shortest_distance
        
    return shortest_distance

def PID(kp=10, kd=0, ki=0.1, total_distance = 15, velocity = 1, show_ki = True, show_kp = True, show_kd = True):
    # create an instance of the graph class
    track = graph(0, 0, 90) # intial position (0, 0) orientation 90 degrees 
    car = graph(0, 0.2, 90)

    track_resolution = 0.1 # higher -> less precise
    def track_forward(distance):
        nonlocal track, track_resolution
        for _ in range(int(distance / track_resolution)):
            track.move_forward(1 * track_resolution)

    def track_turn(direction, angle, radius): # direction -1 right 1 left, angle (degrees) radius of the turn
        nonlocal track, track_resolution
        for _ in range(int(angle / track_resolution)):
            track.turn(direction * track_resolution)
            track.move_forward(np.pi / 180 * radius * track_resolution)
        
    track_forward(5)
    track_turn(-1, 170, 1) # direction = -1 (right) angle = 170 (degrees) radius = 1
    track_forward(5)
    track_turn(1, 170, 1)
    track_forward(5)
    track_turn(-1, 170, 1)
    track_forward(5)
    track_turn(1, 170, 1)
    track_forward(5)
    
    

    omega = 0
    error = 0
    prev_error = error
    error_sum = 0

    distance = 0
    errors = []  # List to store the error at each iteration
    omegas = []  # List to store the omega at each iteration
    contributions = []  # List to store the contributions of kp, ki, kd at each iteration

    while distance < total_distance:
        error = (calculate_error(car, track)) * 0.3 + prev_error * 0.7
        
        error_sum += error
        p = kp * error
        i = ki * error_sum
        d = kd * (error - prev_error)
        omega = p + i + d
        if (omega) > 40:
            omega = 40
        elif (omega) < -40:
            omega = -40
        car.theta += omega
        car.move_forward(velocity * 0.1)
        distance += velocity * 0.1
        prev_error = error
        errors.append(error)  # Append the current error to the list
        omegas.append(omega)  # Append the current omega to the list
        contributions.append((p, i, d))  # Append the current contributions to the list

    graph_resolution = 1 # higher -> less precise
    # Plot the contributions of kp, ki, kd against iteration
    x = range(0, len(contributions), graph_resolution)
    kp = [c[0] for c in contributions][::graph_resolution] if show_kp else [0]*len(x)
    ki = [c[1] for c in contributions][::graph_resolution] if show_ki else [0]*len(x)
    kd = [c[2] for c in contributions][::graph_resolution] if show_kd else [0]*len(x)

    # Create a figure
    plt.figure(figsize=(12, 6))

    # Create the first subplot for the histogram
    plt.subplot(1, 2, 1)  # 1 row, 2 columns, first plot
    sns.histplot(errors, bins=50, color='blue', kde=True, stat='density')
    plt.xlabel('Error')
    plt.ylabel('Density')
    plt.title('Histogram of Errors')

    # Create the second subplot for the scatter plot
    plt.subplot(1, 2, 2)  # 1 row, 2 columns, second plot
    plt.gca().set_aspect('equal')
    plt.grid(True)  # This activates the grid
    
    plt.plot(track.x_points, track.y_points, label="Trajectory")
    plt.plot(car.x_points, car.y_points, label="Car")
    plt.legend()
    plt.xlabel('X-Position')
    plt.ylabel('Y-Position')

    # Show the plots
    plt.tight_layout()  # Adjusts subplot params so that subplots fit into the figure area
    plt.show()

    # Stackplot of kp, ki, kd contributions
    plt.figure()
    plt.stackplot(x, kp, ki, kd, labels=['kp contribution', 'ki contribution', 'kd contribution'])
    plt.plot(x, omegas[::graph_resolution], label="Omega", color = 'blue', linewidth=1, linestyle='-')  # Plot the omega
    plt.legend(loc='upper left')
    plt.xlabel('Time')
    plt.ylabel('Contribution')
    plt.show()


    plt.figure()
    sns.scatterplot(x=errors, y=omegas, color='red', hue=errors, legend=False)
    plt.xlabel('Error')
    plt.ylabel('Omega')
    plt.title('Scatter plot of Error vs Omega')
    plt.show()

    # Box plot of PID contributions
    plt.figure()
    p_contributions, i_contributions, d_contributions = zip(*contributions)
    sns.boxplot(data=[p_contributions, i_contributions, d_contributions])
    plt.xticks([0, 1, 2], ['P', 'I', 'D'])
    plt.ylabel('Contribution')
    plt.title('Box plot of PID contributions')
    plt.show()




In [8]:


widgets.interact(
    PID,
    kp=widgets.FloatText(value=40.0, step=1),
    kd=widgets.FloatText(value=300, step=10),
    ki=widgets.FloatText(value=2.2, step=0.5),
    total_distance=widgets.FloatText(value=40, step=1),
    velocity=widgets.FloatText(value=1, step=0.1),
    show_kp=widgets.Checkbox(value=True, description='Show kp contribution'),
    show_ki=widgets.Checkbox(value=True, description='Show ki contribution'),
    show_kd=widgets.Checkbox(value=True, description='Show kd contribution')
)



interactive(children=(FloatText(value=40.0, description='kp', step=1.0), FloatText(value=300.0, description='k…

<function __main__.PID(kp=10, kd=0, ki=0.1, total_distance=15, velocity=1, show_ki=True, show_kp=True, show_kd=True)>